# bert_score

In [1]:
from bert_score import score
import json

# Load your JSON
with open('test_questions.json', 'r') as f:
    data = json.load(f)

# Extract AI and GT answers
ai_answers = [item['AI'] for item in data]
gt_answers = [item['answer'] for item in data]

# Calculate BERTScore
P, R, F1 = score(ai_answers, gt_answers, lang="en", verbose=True)

# Print average
print(f"Average Precision: {P.mean().item():.4f}")
print(f"Average Recall: {R.mean().item():.4f}")
print(f"Average F1: {F1.mean().item():.4f}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 37.69 seconds, 0.66 sentences/sec
Average Precision: 0.8957
Average Recall: 0.8731
Average F1: 0.8839


### The BERT exibits following result

Average Precision: 0.8957

Average Recall: 0.8731

Average F1: 0.8839

In [11]:
import json
from sentence_transformers import SentenceTransformer, util

# Load your JSON data
with open('test_questions.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Load the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Track total similarity
total_similarity = 0.0
num_items = len(data)

# Compute cosine similarity for each item
for item in data:
    ai_text = item.get("AI", "")
    answer_text = item.get("answer", "")

    # Get embeddings
    ai_embedding = model.encode(ai_text, convert_to_tensor=True)
    answer_embedding = model.encode(answer_text, convert_to_tensor=True)

    # Compute cosine similarity
    similarity = util.cos_sim(ai_embedding, answer_embedding).item()
    total_similarity += similarity

# Calculate average
average_similarity = total_similarity / num_items

print(f"Average cosine similarity across {num_items} items: {average_similarity:.4f}")


Average cosine similarity across 25 items: 0.7012


In [16]:
import json
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from bert_score import score as bert_score

# Load data
with open('test_questions.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Initialize variables
bleu_scores = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

references = []
candidates = []

# Initialize scorers
rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
smoothie = SmoothingFunction().method4

# Calculate metrics
for item in data:
    ref = item['answer']
    cand = item['AI']
    
    # BLEU
    bleu = sentence_bleu([ref.split()], cand.split(), smoothing_function=smoothie)
    bleu_scores.append(bleu)
    
    # ROUGE
    scores = rouge.score(cand, ref)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)
    
    # For BERTScore
    references.append(ref)
    candidates.append(cand)


# Averages
avg_bleu = sum(bleu_scores) / len(bleu_scores)
avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

# Print results
print(f"Average BLEU Score: {avg_bleu:.4f}")
print(f"Average ROUGE-1 F1: {avg_rouge1:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL:.4f}")



Average BLEU Score: 0.0573
Average ROUGE-1 F1: 0.3462
Average ROUGE-2 F1: 0.1380
Average ROUGE-L F1: 0.2348
